In [93]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

# root path
ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the project root to the Python path
if ROOT not in sys.path:
    sys.path.append(ROOT)

In [94]:
def save_earnings_expenses_plot(df: pd.DataFrame):
    """
    Generate a bar plot for earnings and expenses using pastel blue shades.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing the summary data with columns for earnings and expenses.
    """
    # Define two shades of pastel blue
    pastel_blues = ['#ACC6DE', '#4A6A94']  # Light pastel blue and slightly darker pastel blue
    
    # Create the Bar Plot with improved styling
    fig, ax = plt.subplots(figsize=(12, 7))
    bars = df.plot(kind='bar', ax=ax, color=pastel_blues)
    
    # Enhance the plot styling
    plt.title('Earnings and Expenses', fontsize=16, pad=20, fontweight='bold')
    plt.ylabel('Amount', fontsize=12)
    plt.xticks(ticks=[0], labels=["Total"], rotation=0)
    
    # Add grid for better readability
    ax.yaxis.grid(True, linestyle='--', alpha=0.7)
    ax.set_axisbelow(True)
    
    # Add value labels on top of each individual bar
    for i, column in enumerate(df.columns):
        value = df[column].values[0]
        ax.text(0 + (i-0.5)*0.25, value, f'${value:,.0f}', 
                ha='center', va='bottom', fontweight='bold')
    
    # Customize the frame
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # Define the path and ensure the directory exists
    figures_dir = os.path.join(ROOT, 'reports', 'figures')
    os.makedirs(figures_dir, exist_ok=True)
    plot_path = os.path.join(figures_dir, 'earnings_and_expenses.png')
    
    # Save the plot with higher DPI for better quality
    fig.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close(fig)
    print(f"Earnings and expenses plot saved at {plot_path}")

def save_expenses_summary_plot(df: pd.DataFrame):
    """
    Generate a bar plot for expenses by merchant category using a single pastel color.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing the summary data with columns 'Expenses Type' and 'Total Amount'.
    """
    # Define single pastel color for all bars
    pastel_color = '#4A6A94'  # Light pastel blue
    
    # Create the Bar Plot with improved styling
    fig, ax = plt.subplots(figsize=(12, 7))
    bars = df.plot.bar(x='Expenses Type', y='Total Amount', ax=ax, 
                      color=pastel_color, 
                      width=0.7)
    
    # Enhance the plot styling
    plt.title('Expenses by Merchant Category', fontsize=16, pad=20, fontweight='bold')
    plt.ylabel('Total Amount', fontsize=12)
    plt.xlabel('', fontsize=12)  # Remove x-label as it's self-explanatory
    
    # Rotate labels for better readability
    plt.xticks(rotation=45, ha='right')
    
    # Add grid for better readability
    ax.yaxis.grid(True, linestyle='--', alpha=0.7)
    ax.set_axisbelow(True)
    
    # Add value labels on top of bars
    for i, v in enumerate(df['Total Amount']):
        ax.text(i, v, f'${v:,.0f}', ha='center', va='bottom', fontweight='bold')
    
    # Customize the frame
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    plt.tight_layout()
    
    # Define the path and ensure the directory exists
    figures_dir = os.path.join(ROOT, 'reports', 'figures')
    os.makedirs(figures_dir, exist_ok=True)
    plot_path = os.path.join(figures_dir, 'expenses_summary.png')
    
    # Save the plot with higher DPI for better quality
    fig.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close(fig)
    print(f"Expenses summary plot saved at {plot_path}")

def earnings_and_expenses(
    df: pd.DataFrame, client_id: int, start_date: str, end_date: str
) -> pd.DataFrame:
    """
    For the period defined in between start_date and end_date (both included), get the client data available and return
    a pandas DataFrame with the Earnings and Expenses total amount for the period range and user given.The expected columns are:
        - Earnings
        - Expenses
    The DataFrame should have the columns in this order ['Earnings','Expenses']. Round the amounts to 2 decimals.

    Create a Bar Plot with the Earnings and Expenses absolute values and save it as "reports/figures/earnings_and_expenses.png" .

    Parameters
    ----------
    df : pandas DataFrame
       DataFrame of the data to be used for the agent.
    client_id : int
        Id of the client.
    start_date : str
        Start date for the date period. In the format "YYYY-MM-DD".
    end_date : str
        End date for the date period. In the format "YYYY-MM-DD".


    Returns
    -------
    Pandas Dataframe with the earnings and expenses rounded to 2 decimals.

    """
    
    # df = clean_transactions_df(df)

    # Filter data for the client and date range
    mask = (
        (df['client_id'] == client_id) &
        (df['date'] >= start_date) &
        (df['date'] <= end_date)
    )
    client_data = df.loc[mask]

    # Return an empty DataFrame if no data is available for the client in the date range
    if client_data.empty:
        print("No data available for the specified client ID and date range.")
        # Prepare the DataFrame
        zero_result_df = pd.DataFrame(columns=['Earnings', 'Expenses'])
        # save_earnings_expenses_plot(zero_result_df)
        return zero_result_df

    # Calculate Earnings and Expenses
    earnings = client_data[client_data['amount_clean'] > 0]['amount_clean'].sum()
    expenses = client_data[client_data['amount_clean'] < 0]['amount_clean'].sum()

    # Prepare the DataFrame
    data = {
        'Earnings': [round(earnings, 2)],
        'Expenses': [round(expenses, 2)]
    }
    result_df = pd.DataFrame(data)
    save_earnings_expenses_plot(result_df)

    return result_df

def expenses_summary(
    df: pd.DataFrame, client_id: int, start_date: str, end_date: str
) -> pd.DataFrame:
    """
    For the period defined in between start_date and end_date (both included), get the client data available and return
    a Pandas Data Frame with the Expenses by merchant category. The expected columns are:
        - Expenses Type --> (merchant category names)
        - Total Amount
        - Average
        - Max
        - Min
        - Num. Transactions
    The DataFrame should be sorted alphabeticaly by Expenses Type and values have to be rounded to 2 decimals. Return the dataframe with the columns in the given order.
    The merchant category names can be found in data/raw/mcc_codes.json .

    Create a Bar Plot with the data in absolute values and save it as "reports/figures/expenses_summary.png" .

    Parameters
    ----------
    df : pandas DataFrame
       DataFrame  of the data to be used for the agent.
    client_id : int
        Id of the client.
    start_date : str
        Start date for the date period. In the format "YYYY-MM-DD".
    end_date : str
        End date for the date period. In the format "YYYY-MM-DD".

    Returns
    -------
    Pandas Dataframe with the Expenses by merchant category.

    """

    # Load merchant category codes
    mcc_codes_path = os.path.join(ROOT, 'data', 'raw', 'mcc_codes.json')
    with open(mcc_codes_path, 'r') as f:
        mcc_codes = json.load(f)

    # df = clean_transactions_df(df)

    # Filter data for the client and date range
    mask = (
        (df['client_id'] == client_id) &
        (df['date'] >= start_date) &
        (df['date'] <= end_date) &
        (df['amount_clean'] < 0)  # Only expenses
    )
    client_data = df.loc[mask]

    # Return an empty DataFrame if no data is available for the client in the date range
    if client_data.empty:
        print("No data available for the specified client ID and date range.")
        zero_result_df = pd.DataFrame(columns=['Expenses Type', 'Total Amount', 'Average', 'Max', 'Min', 'Num. Transactions'])
        
        # save_expenses_summary_plot(zero_result_df)
        return zero_result_df

    client_data = client_data.copy()
    client_data.loc[:, 'Expenses Type'] = client_data['mcc'].astype(str).map(mcc_codes)
    client_data.loc[:, 'Expenses Type'] = client_data['Expenses Type'].fillna('Unknown')
    client_data.loc[:, 'amount_clean'] = client_data['amount_clean'].abs()

    summary = client_data.groupby('Expenses Type')['amount_clean'].agg([
        ('Total Amount', lambda x: round(x.sum(), 2)),
        ('Average', lambda x: round(x.mean(), 2)),
        ('Max', lambda x: round(x.min(), 2)),  # min because amounts are negative
        ('Min', lambda x: round(x.max(), 2)),  # max because amounts are negative
        ('Num. Transactions', 'count')
    ]).reset_index()

    # Sort alphabetically by 'Expenses Type'
    summary.sort_values('Expenses Type', inplace=True)
    save_expenses_summary_plot(summary)

    return summary

def cash_flow_summary(
    df: pd.DataFrame, client_id: int, start_date: str, end_date: str
) -> pd.DataFrame:
    """
    For the period defined by start_date and end_date (both inclusive), retrieve the available client data and return a Pandas DataFrame containing cash flow information.

    If the period exceeds 60 days, group the data by month, using the end of each month for the date. If the period is 60 days or shorter, group the data by week.

        The expected columns are:
            - Date --> the date for the period. YYYY-MM if period larger than 60 days, YYYY-MM-DD otherwise.
            - Inflows --> the sum of the earnings (positive amounts)
            - Outflows --> the sum of the expenses (absolute values of the negative amounts)
            - Net Cash Flow --> Inflows - Outflows
            - % Savings --> Percentage of Net Cash Flow / Inflows

        The DataFrame should be sorted by ascending date and values rounded to 2 decimals. The columns should be in the given order.

        Parameters
        ----------
        df : pandas DataFrame
           DataFrame  of the data to be used for the agent.
        client_id : int
            Id of the client.
        start_date : str
            Start date for the date period. In the format "YYYY-MM-DD".
        end_date : str
            End date for the date period. In the format "YYYY-MM-DD".


        Returns
        -------
        Pandas Dataframe with the cash flow summary.

    """

    # Calculate the period length
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)
    period_length = (end - start).days

    # df = clean_transactions_df(df)

    # Filter data for the client and date range
    mask = (
        (df['client_id'] == client_id) &
        (df['date'] >= start) &
        (df['date'] <= end)
    )
    client_data = df.loc[mask]

    # Return an empty DataFrame if no data is available for the client in the date range
    if client_data.empty:
        print("No data available for the specified client ID and date range.")
        # Prepare a simple DataFrame with zero values
        zero_result_df = pd.DataFrame(columns=['Date', 'Inflows', 'Outflows', 'Net Cash Flow', '% Savings'])
        return zero_result_df

    # Determine grouping frequency
    if period_length > 60:
        freq = 'ME'
        date_format = '%Y-%m'
    else:
        freq = 'W'
        date_format = '%Y-%m-%d'

    # Group and aggregate data
    client_data.set_index('date', inplace=True)
    grouped = client_data.groupby(pd.Grouper(freq=freq))

    summary = grouped['amount_clean'].agg([
        ('Inflows', lambda x: x[x > 0].sum()),
        ('Outflows', lambda x: abs(x[x < 0].sum()))
    ])

    summary['Net Cash Flow'] = summary['Inflows'] - summary['Outflows']
    summary['% Savings'] = (summary['Net Cash Flow'] / summary['Inflows']) * 100
    summary.reset_index(inplace=True)

    # Format the 'Date' column
    summary['Date'] = summary['date'].dt.strftime(date_format)
    summary.drop('date', axis=1, inplace=True)

    # Round values to 2 decimals
    summary = summary.round(2)

    # Sort by Date
    summary.sort_values('Date', inplace=True)

    return summary[['Date', 'Inflows', 'Outflows', 'Net Cash Flow', '% Savings']]

In [95]:
data = pd.read_pickle(os.path.join(ROOT, 'data', 'processed', 'transactions_df.pkl')).rename(columns={'amount': 'amount_clean'})  

In [96]:
client = 126
start = "2013-01-01"
end = "2020-01-31"

In [97]:
earnings_and_expenses(data, client, start, end)
#[176279.33], "Expenses": [-18246.0]}

Earnings and expenses plot saved at /home/ezemriv/git_projects/AI-Agent-Report-Maker-Full/reports/figures/earnings_and_expenses.png


,Earnings,Expenses
0,176279.34375,-18246.0


In [98]:
expenses_summary(data, client, start, end)

Expenses summary plot saved at /home/ezemriv/git_projects/AI-Agent-Report-Maker-Full/reports/figures/expenses_summary.png


,Expenses Type,Total Amount,Average,Max,Min,Num. Transactions
0,"Electroplating, Plating, Polishing Services",426.0,426.000000,426.0,426.0,1
1,Gardening Supplies,446.0,446.000000,446.0,446.0,1
2,Heat Treating Metal Services,457.0,457.000000,457.0,457.0,1
3,"Lighting, Fixtures, Electrical Supplies",477.0,477.000000,477.0,477.0,1
4,Miscellaneous Food Stores,6871.0,75.510002,50.0,99.0,91
5,Miscellaneous Machinery and Parts Manufacturing,156.0,156.000000,156.0,156.0,1
6,Non-Ferrous Metal Foundries,179.0,179.000000,179.0,179.0,1
7,Railroad Passenger Transport,437.0,437.000000,437.0,437.0,1
8,Service Stations,8241.0,73.580002,51.0,100.0,112
9,Ship Chandlers,556.0,278.000000,225.0,331.0,2


In [99]:
expenses_summary(data, 32, "2011-05-01", "2016-11-30")

Expenses summary plot saved at /home/ezemriv/git_projects/AI-Agent-Report-Maker-Full/reports/figures/expenses_summary.png


,Expenses Type,Total Amount,Average,Max,Min,Num. Transactions
0,Computer Network Services,1161.0,290.250000,104.0,420.0,4
1,"Electroplating, Plating, Polishing Services",856.0,285.329987,102.0,476.0,3
2,Gardening Supplies,1592.0,398.000000,367.0,469.0,4
3,Heat Treating Metal Services,385.0,385.000000,385.0,385.0,1
4,Industrial Equipment and Supplies,244.0,244.000000,244.0,244.0,1
5,"Lighting, Fixtures, Electrical Supplies",1530.0,306.000000,211.0,410.0,5
6,"Lodging - Hotels, Motels, Resorts",1314.0,328.500000,157.0,489.0,4
7,Miscellaneous Food Stores,6149.0,74.989998,50.0,100.0,82
8,Miscellaneous Metalwork,728.0,364.000000,283.0,445.0,2
9,Non-Ferrous Metal Foundries,1185.0,395.000000,328.0,495.0,3


In [100]:
client = 126
start = "2013-01-01"
end="2013-02-28"

cash_flow_summary(data, client, start, end)

,Date,Inflows,Outflows,Net Cash Flow,% Savings
0,2013-01-06,432.739990,129.0,303.739990,70.190002
1,2013-01-13,518.669983,0.0,518.669983,100.000000
2,2013-01-20,297.630005,66.0,231.630005,77.820000
3,2013-01-27,450.910004,0.0,450.910004,100.000000
4,2013-02-03,863.159973,156.0,707.159973,81.930000
5,2013-02-10,288.750000,0.0,288.750000,100.000000
6,2013-02-17,466.820007,0.0,466.820007,100.000000
7,2013-02-24,1019.020020,71.0,948.020020,93.029999
8,2013-03-03,138.350006,0.0,138.350006,100.000000
